#### 1. Setup and Importing Libraries

In [48]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset # wraps an iterable around the dataset
from torchvision import datasets    # stores the samples and their corresponding labels
from torchvision.transforms import transforms  # transformations we can perform on our dataset
from torchvision.transforms import ToTensor
import pandas as pd
import numpy as np
import os
import wandb
import matplotlib.pyplot as plt

In [49]:
# Set API Key
os.environ["WANDB_API_KEY"] = "cf61e02cee13abdd3d8a232d29df527bd6cc7f89"

# Set the WANDB_NOTEBOOK_NAME environment variable to the name of your notebook (manually)
os.environ["WANDB_NOTEBOOK_NAME"] = "DataLoader.ipynb"

# set the WANDB_TEMP environment variable to a directory where we have write permissions
os.environ["WANDB_TEMP"] = os.getcwd()
os.environ["WANDB_DIR"] = os.getcwd()
os.environ["WANDB_CONFIG_DIR"] = os.getcwd()

In [50]:
wandb.init(project='ECG-analysis-with-Deep-Learning-on-GPU-accelerators')

In [51]:
# Get cpu, gpu or mps device for training 
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [52]:
#torch.cuda.get_device_name(0)

#### 2. Data Loader

In [53]:
class ECGDataSet(Dataset):
    
    def __init__(self, split='train'):

        self.split = split

        # data loading
        current_directory = os.getcwd()
        self.parent_directory = os.path.dirname(current_directory)
        train_small_path = os.path.join(self.parent_directory, 'data', 'deepfake-ecg-small', str(self.split) + '.csv')
        self.df = pd.read_csv(train_small_path)  # Skip the header row
        
        # Avg RR interval
        # in milli seconds
        RR = torch.tensor(self.df['avgrrinterval'].values, dtype=torch.float32)
        # calculate HR
        self.y = 60 * 1000/RR

        # Size of the dataset
        self.samples = self.df.shape[0]

    def __getitem__(self, index):
        
        # file path
        filename= self.df['patid'].values[index]
        asc_path = os.path.join(self.parent_directory, 'data', 'deepfake-ecg-small', str(self.split), str(filename) + '.asc')
        
        ecg_signals = pd.read_csv( asc_path, header=None, sep=" ") # read into dataframe
        ecg_signals = torch.tensor(ecg_signals.values) # convert dataframe values to tensor
        
        ecg_signals = ecg_signals.float()
        
        # Transposing the ecg signals
        ecg_signals = ecg_signals/6000 # normalization
        ecg_signals = ecg_signals.t() 
        
        qt = self.y[index]
        # Retrieve a sample from x and y based on the index
        return ecg_signals, qt

    def __len__(self):
        # Return the total number of samples in the dataset
        return self.samples
    

In [54]:
# ECG dataset
train_dataset = ECGDataSet(split='train')
validate_dataset = ECGDataSet(split='validate')

In [55]:
# first data
first_data = train_dataset[0]
x, y = first_data

In [56]:
x

tensor([[-0.0212, -0.0270, -0.0237,  ..., -0.0148, -0.0065, -0.0155],
        [-0.0002,  0.0000, -0.0077,  ..., -0.0030,  0.0037,  0.0008],
        [-0.0055, -0.0013, -0.0045,  ...,  0.0073,  0.0118,  0.0137],
        ...,
        [-0.0153, -0.0143, -0.0145,  ...,  0.0112,  0.0148,  0.0175],
        [-0.0102, -0.0112, -0.0117,  ...,  0.0087,  0.0147,  0.0043],
        [ 0.0003, -0.0048, -0.0042,  ...,  0.0115,  0.0213,  0.0192]])

In [57]:
y

tensor(59.6421)

In [58]:
x.shape

torch.Size([8, 5000])

In [59]:
y.shape

torch.Size([])

### Residual Convoluted Neural Network

In [60]:
# data loader
# It allows you to efficiently load and iterate over batches of data during the training or evaluation process.
train_dataloader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True, num_workers=2)
validate_dataloader = DataLoader(dataset=validate_dataset, batch_size=32, shuffle=True, num_workers=2)

In [61]:
for x,y in train_dataloader:
    print(x.shape, y.shape)
    print(x.dtype, y.dtype)
    break

torch.Size([32, 8, 5000]) torch.Size([32])
torch.float32 torch.float32


In [62]:
# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        # First convolutional layer of the residual block
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.relu = nn.ReLU()
        # Second convolutional layer of the residual block
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)

    def forward(self, x):
        residual = x
        # Pass input through the first convolutional layer
        out = self.conv1(x)
        out = self.relu(out)
        # Pass the output of the first convolutional layer through the second convolutional layer
        out = self.conv2(out)
        # Add the residual connection
        out += residual
        out = self.relu(out)
        return out

In [63]:
# Residual CNN model
class ResidualCNN(nn.Module):
    def __init__(self, num_classes):
        super(ResidualCNN, self).__init__()
        # Initial convolutional layer
        self.conv1 = nn.Conv1d(8, 16, kernel_size=2, stride=1, padding=1)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool1d(kernel_size=2, stride=2)
        # First residual block
        self.res_block1 = ResidualBlock(16, 16)
        # Second residual block
        self.res_block2 = ResidualBlock(16, 16)
        # Fully connected layer
        self.fc = nn.Linear(16 * 2500, num_classes)

    def forward(self, x):
        # Pass input through the initial convolutional layer
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        # Pass the output through the first residual block
        x = self.res_block1(x)
        # Pass the output through the second residual block
        x = self.res_block2(x)
        x = self.relu(x)
        x = x.view(x.size(0), -1)
        # Pass the flattened output through the fully connected layer
        x = self.fc(x)
        return x

In [64]:
# hyperparameters
num_classes = 1  # Number of output classes
num_epochs = 100
learning_rate = 0.000001

In [65]:
wandb.config.num_epochs = num_epochs
wandb.config.learning_rate = learning_rate

In [66]:
model = ResidualCNN(num_classes)

# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()

# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [67]:
# Set up the wandb configuration and log hyperparameters
wandb.config.num_epochs = num_epochs
wandb.config.learning_rate = learning_rate

In [68]:
def MAE(losses):
    error_sum = 0
    for loss in losses:
        absolute_error = abs(loss - 0)  # Assuming 0 is the target value
        error_sum += absolute_error

    mean_absolute_error = error_sum / len(losses)
    return mean_absolute_error

In [69]:
%%time

train_losses = []
val_losses = []
epochs = []

for epoch in range(wandb.config.num_epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    epochs.append(epoch)

    train_losses_epoch = [] 
    for batch_inputs, batch_labels in train_dataloader:

        # Forward pass
        outputs = model(batch_inputs)
        loss = criterion(outputs, batch_labels)
        train_losses_epoch.append(int(loss))

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    
    train_loss = MAE(train_losses_epoch)
    train_losses.append(train_loss)


    model.eval()
    with torch.no_grad():
        val_losses_epoch = []  # List to store validation losses for the current epoch
        for batch, (X_val, y_val) in enumerate(validate_dataloader):
            #X_val, y_val = X_val.to(device), y_val.to(device)

            val_pred = model(X_val)
            val_loss = criterion(val_pred, y_val)

            val_losses_epoch.append(int(val_loss))

        val_loss = MAE(val_losses_epoch)
        val_losses.append(val_loss)

wandb.log({"ResNet: loss [mean absolute error] vs epoch" : wandb.plot.line_series(
                       xs=epochs, 
                       ys=[train_losses, val_losses],
                       keys=["training", "validation"],
                       title="",
                       xname="epochs")})

print("Done!")

Epoch 1
-------------------------------


/storage/projects2/e17-4yp-compreh-ecg-analysis/minicondaInst/envs/test/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/storage/projects2/e17-4yp-compreh-ecg-analysis/minicondaInst/envs/test/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 2
-------------------------------


/storage/projects2/e17-4yp-compreh-ecg-analysis/minicondaInst/envs/test/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([4, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 3
-------------------------------
Epoch 4
-------------------------------
Epoch 5
-------------------------------
Epoch 6
-------------------------------
Epoch 7
-------------------------------
Epoch 8
-------------------------------
Epoch 9
-------------------------------
Epoch 10
-------------------------------
Epoch 11
-------------------------------
Epoch 12
-------------------------------
Epoch 13
-------------------------------
Epoch 14
-------------------------------
Epoch 15
-------------------------------
Epoch 16
-------------------------------
Epoch 17
-------------------------------
Epoch 18
-------------------------------
Epoch 19
-------------------------------
Epoch 20
-------------------------------
Epoch 21
-------------------------------
Epoch 22
-------------------------------
Epoch 23
-------------------------------
Epoch 24
-------------------------------
Epoch 25
-------------------------------
Epoch 26
-------------------------------
Epoch 27
--------------

In [70]:
# finish
wandb.finish()